# Generate Mid using dedicated service - Python

### Overview
Mid indicators are a commonly used to provide midpoint value from two given inputs.

This sample demonstrates how to request and plot from a dedicated data service the following midpoints:
1. Midprice
2. Midsize

When correlated with a range of indicators, it gives an idea about the market quality.

### Inputs/outputs
Mid indicator requires instrument's identifier and date time intervals as per inputs and returns the midpoints' values.

For each timestamp, the midpoint's value is an average of:
* bid and ask prices for midprice
* bid and ask sizes for midsize

This sample shows how to plot a simple mid graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve top of the book tick data from the hosted service. The queried endpoint in this script are:
* *TickBookService*: to directly retrieve best Bid/Ask tick data from the server.

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *matpotlib* and *plotly* as per display package

***

# Run mid sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis==2.36.* --pre

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick.v1.tick_book_pb2 as tick_book
import systemathics.apis.type.shared.v1.book_updates_pb2 as book_updates
import systemathics.apis.services.tick.v1.tick_book_pb2_grpc as tick_book_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *tick book* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick book request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day = 5), 
    end_date = date.Date(year = 2021, month = 11, day = 5)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 14, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 14, minutes = 30, seconds = 0)
)

#### 3.3 Request creation
The following code snippets creates *gRPC client*, process request and returns the request reply:

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate tick book request
my_identifier = identifier.Identifier(exchange = exchange, ticker = ticker)
request = tick_book.TickBookRequest(
    identifiers = [my_identifier],
    constraints = my_constraints,
    book_updates = book_updates.BOOK_UPDATES_SNAPSHOTS_ONLY 
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick book service
        service = tick_book_service.TickBookServiceStub(channel)
        
        # process the tick trades request
        limits_data = []
        metadata = [('authorization', token)]
        for book in service.TickBook(request=request, metadata=metadata):
            if book.data:
                limits_data.append(book.data)

    # keep only limits, delete instrument mapping (1st element retrieved)
    # mapping is not used as only one instrument identifier is requested
    del limits_data[0]
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved top of the book limits count:

In [ ]:
print('Total top of the book limits retrieved: {0}'.format(len(limits_data)))

### Step 4: Visualize data

#### 4.1 Retrieve mid data
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates = [datetime.fromtimestamp(l.time_stamp.seconds) for l in limits_data]
mid_prices = [(l.book.ask[0].price + l.book.bid[0].price) / 2  for l in limits_data]

In [ ]:
# create a pandas dataframe with: dates and mids
d = {'Date': dates, 'Mid price': mid_prices}
df = pd.DataFrame(data=d)
df

#### 4.2 Plot Mid with matplotlib

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(1,1,figsize=(25,10))
ax.plot( 'Date', 'Mid price', data=df, marker='', color='blue', linewidth=1, label="Mid price")
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Mid price",color="black",fontsize=14)
plt.legend()
plt.title('Mid price for {0}-{1}'.format(ticker, exchange))
plt.show()

#### 4.3 Plot Mid with matplotlib

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Mid price'], name='Mid price', marker_color ='blue'))
fig.update_xaxes(title_text = 'Date')
fig.update_layout(title = 'Mid price for {0}-{1}'.format(ticker,exchange))
fig.show()